<a href="https://colab.research.google.com/github/Rstam59/TaskDataRepoForStudents/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # This loads everything from .env into os.environ


In [ ]:
import os
print(os.environ.get("LANGCHAIN_API_KEY"))  # should print your key


In [ ]:
import os

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] =  #Its not safe should have used export LANGCHAIN_API_KEY="your-key"


In [ ]:
os.environ['OPENAI_API_KEY'] =

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ('post-content', 'post-title', 'post-header')
        )
    )
)


docs = loader.load()

#Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200
)
splits = text_splitter.split_documents(docs)


#Embed
vectorstore = Chroma.from_documents(
    documents = splits,
    embedding = OpenAIEmbeddings()
)


retriever = vectorstore.as_retriever()


#prompt
prompt = hub.pull("rlm/rag-prompt")

#llm
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0)

#Post-processing
def format_docs(docs):
    "/n/n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("what is task decomposition?")

# Part 2: Indexing

In [ ]:
question = 'What kind of pets do i like?'
document = 'My favorite pet is a cat.'

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


num_tokens_from_string(question, 'cl100k_base')

In [ ]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result), len(document_result)

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)


similarity = cosine_similarity(query_result, document_result)
similarity

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths = ('https://lilianweng.github.io/posts/2023-06-23-agent/',),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ('post-content', 'post-title', 'post-header')
        )
    )
)

blog_docs = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300, chunk_overlap = 50
)

splits = text_splitter.split_documents(blog_docs)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = splits,
    embedding = OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

#Retrieval

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents = splits,
                                    embedding = OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs = {'k': 1})

In [ ]:
docs = retriever.invoke(input = 'what is task decomposition')

In [ ]:
len(docs)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# Chain
chain = prompt | llm

In [ ]:
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

In [ ]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [ ]:
prompt_hub_rag

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

# Part 5: Multi Query

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature = 0)
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [ ]:
print("Generated Queries:", generate_queries.invoke({'question': question}))


In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

question = 'What is task decomposition for LLM agents?'
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({'question': question})
len(docs)

In [ ]:
queries = generate_queries.invoke({'question': question})
print("Generated Queries:\n", "\n".join(queries))

docs_per_query = retriever.map().invoke(queries)
print("\nDocs retrieved per query:")
for i, doclist in enumerate(docs_per_query):
    print(f"\nQuery {i+1}: '{queries[i]}'")
    for doc in doclist:
        print(f"  → {doc.metadata}, preview: {doc.page_content[:100]}")



In [ ]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough


#rag
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature = 0)

final_rag_chain = (
    {'context': retrieval_chain,
     'question': itemgetter('question')}
    | prompt
    | llm
    | StrOutputParser()
)


final_rag_chain.invoke({'question': question})

#Part 6: RAG-Fusion

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature = 0)
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k = 60):
    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)

            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0

            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_result = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key = lambda x: x[1], reverse = True)
    ]
    return reranked_result

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({'question': question})
len(docs)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

template = """Answer the following question based on this conext:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {'context': retrieval_chain_rag_fusion,
     'question': itemgetter('question')}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({'question': question})